In [3]:
from typing import List
from typing import Self
import numpy as np

In [2]:
# 定数定義
NUMBER_OF_EDGE = 9
NUMBER_OF_SQUARES = 9 ** 2
MAXIMUM_NUMBER_OF_DIRECTIONS_DUE_TO_POSITIONAL_MOVEMENT = 2
NUMBER_OF_DIRECTIONS = 8
NUMBER_OF_PIECES = 40

In [3]:
# Enum
from enum import IntEnum, Enum

class Direction(IntEnum):
    UP = 0
    UP_LEFT = 1
    LEFT = 2
    DOWN_LEFT = 3
    DOWN = 4
    DOWN_RIGHT = 5
    RIGHT = 6
    UP_RIGHT = 7

class PieceName(IntEnum):
    NON = -1
    KING = 0
    ROOK = 1
    BICHOP = 2
    GOLDGENERAL = 3
    SILVERGENERAL = 4
    KNIGHT = 5
    LANCE = 6
    PAWN = 7
    PROMOTED_ROOK = 8
    PROMOTED_BICHOP = 9
    PROMOTED_SILVERGENERAL = 10
    PROMOTED_KNIGHT = 11
    PROMOTED_LANCE = 12
    PROMOTED_PAWN = 13

class Promotion(IntEnum):
    PROMOTABLE = True
    NOTPROMOTABLE = False

class Turn(IntEnum):
    FIRST = True
    SECOND = False

class TemplateMove(IntEnum):
    NEXTTO = 0
    UNLIMITED = 1
    HOP = 2
    NOT = 3

class SquareStatus(IntEnum):
    NOT_USED = False
    USED = True

class ConsoleColor:
    FIRST = '\x1b[36m'
    SECOND = '\x1b[91m'
    RESET = '\x1b[0m'

In [4]:
def switchDirection(direction: Direction) -> Direction:
    return (direction + 4) % 8

In [5]:
class ADirectionMove:
    direction: Direction
    maxLength: int

    def __init__(self, direction: Direction, maxLength: int):
        self.direction = direction
        self.maxLength = maxLength

In [6]:
class WayOfMove:
    through: bool
    adirectionMove: List[ADirectionMove] = [None] * MAXIMUM_NUMBER_OF_DIRECTIONS_DUE_TO_POSITIONAL_MOVEMENT

    def __init__(self, templateMove: TemplateMove, direction: Direction):
        if templateMove == TemplateMove.NEXTTO:
            self.through = False
            self.adirectionMove[0] = ADirectionMove(direction, 1)
            self.adirectionMove[1] = ADirectionMove(Direction.UP, 0)
        elif templateMove == TemplateMove.UNLIMITED:
            self.through = False
            self.adirectionMove[0] = ADirectionMove(direction, NUMBER_OF_EDGE - 1)
            self.adirectionMove[1] = ADirectionMove(Direction.UP, 0)
        elif templateMove == TemplateMove.HOP:
            self.through = True
            self.adirectionMove[0] = ADirectionMove(direction, 1)
            self.adirectionMove[1] = ADirectionMove(Direction.UP, 1)
        elif templateMove == TemplateMove.NOT:
            self.through = False
            self.adirectionMove[0] = ADirectionMove(direction, 0)
            self.adirectionMove[1] = ADirectionMove(Direction.UP, 0)

In [7]:
class Address:
    row: int
    column: int

    def __init__(self, row: int, column: int):
        self.row = row
        self.column = column

    def equal(self, target: Self) -> bool:
        return self.row == target.row and self.column == target.column

In [8]:
def getAddressOfDirection(direction: Direction) -> Address:
    if direction == Direction.UP:
        return Address(-1, 0)
    elif direction == Direction.UP_LEFT:
        return Address(-1, 1)
    elif direction == Direction.LEFT:
        return Address(0, 1)
    elif direction == Direction.DOWN_LEFT:
        return Address(1, 1)
    elif direction == Direction.DOWN:
        return Address(1, 0)
    elif direction == Direction.DOWN_RIGHT:
        return Address(1, -1)
    elif direction == Direction.RIGHT:
        return Address(0, -1)
    elif direction == Direction.UP_RIGHT:
        return Address(-1, -1)

In [9]:
def getPromotion(name: PieceName) -> Promotion:
    if name == PieceName.ROOK or name == PieceName.BICHOP \
        or name == PieceName.SILVERGENERAL or name == PieceName.KNIGHT \
            or name == PieceName.LANCE or name == PieceName.PAWN:
        return Promotion.PROMOTABLE
    else:
        return Promotion.NOTPROMOTABLE

class AbstractPiece:
    name: PieceName
    promotion: Promotion
    wayOfMove: List[WayOfMove] = [None] * NUMBER_OF_DIRECTIONS

    def __init__(self, name: PieceName):
        self.name = name
        self.promotion = getPromotion(name)

        if name == PieceName.NON:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NOT, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NOT, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NOT, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NOT, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NOT, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.UP_RIGHT)
        elif name == PieceName.KING:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
        elif name == PieceName.ROOK:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.UNLIMITED, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NOT, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.UNLIMITED, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.UNLIMITED, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.UNLIMITED, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.UP_RIGHT)
        elif name == PieceName.BICHOP:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NOT, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.UNLIMITED, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NOT, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.UNLIMITED, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NOT, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.UNLIMITED, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NOT, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.UNLIMITED, Direction.UP_RIGHT)
        elif name == PieceName.GOLDGENERAL:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
        elif name == PieceName.SILVERGENERAL:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NOT, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NOT, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NOT, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
        elif name == PieceName.KNIGHT:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NOT, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.HOP, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NOT, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NOT, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NOT, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.HOP, Direction.UP_RIGHT)
        elif name == PieceName.LANCE:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.UNLIMITED, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NOT, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NOT, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NOT, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NOT, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.UP_RIGHT)
        elif name == PieceName.PAWN:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NOT, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NOT, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NOT, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NOT, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.UP_RIGHT)
        elif name == PieceName.PROMOTED_ROOK:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.UNLIMITED, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.UNLIMITED, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.UNLIMITED, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.UNLIMITED, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
        elif name == PieceName.PROMOTED_BICHOP:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.UNLIMITED, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.UNLIMITED, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.UNLIMITED, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.UNLIMITED, Direction.UP_RIGHT)
        elif name == PieceName.PROMOTED_SILVERGENERAL:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
        elif name == PieceName.PROMOTED_KNIGHT:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
        elif name == PieceName.PROMOTED_LANCE:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
        elif name == PieceName.PROMOTED_PAWN:
            self.wayOfMove[Direction.UP] = WayOfMove(TemplateMove.NEXTTO, Direction.UP)
            self.wayOfMove[Direction.UP_LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_LEFT)
            self.wayOfMove[Direction.LEFT] = WayOfMove(TemplateMove.NEXTTO, Direction.LEFT)
            self.wayOfMove[Direction.DOWN_LEFT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_LEFT)
            self.wayOfMove[Direction.DOWN] = WayOfMove(TemplateMove.NEXTTO, Direction.DOWN)
            self.wayOfMove[Direction.DOWN_RIGHT] = WayOfMove(TemplateMove.NOT, Direction.DOWN_RIGHT)
            self.wayOfMove[Direction.RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.RIGHT)
            self.wayOfMove[Direction.UP_RIGHT] = WayOfMove(TemplateMove.NEXTTO, Direction.UP_RIGHT)
    
    def pieceToString(self) -> str:
        if self.name == PieceName.NON:
            return " "
        elif self.name == PieceName.KING:
            return "k"
        elif self.name == PieceName.ROOK:
            return "r"
        elif self.name == PieceName.BICHOP:
            return "b"
        elif self.name == PieceName.GOLDGENERAL:
            return "g"
        elif self.name == PieceName.SILVERGENERAL:
            return "s"
        elif self.name == PieceName.KNIGHT:
            return "n"
        elif self.name == PieceName.LANCE:
            return "l"
        elif self.name == PieceName.PAWN:
            return "p"
        elif self.name == PieceName.PROMOTED_ROOK:
            return "D"
        elif self.name == PieceName.PROMOTED_BICHOP:
            return "H"
        elif self.name == PieceName.PROMOTED_SILVERGENERAL:
            return "S"
        elif self.name == PieceName.PROMOTED_KNIGHT:
            return "N"
        elif self.name == PieceName.PROMOTED_LANCE:
            return "L"
        elif self.name == PieceName.PROMOTED_PAWN:
            return "P"


In [10]:
class Piece:
    piece: AbstractPiece
    index: int
    turn: Turn

    def __init__(self, name: PieceName, index: int, turn: Turn):
        self.piece = AbstractPiece(name)
        self.index = index
        self.turn = turn

    def getBinaryFromPiece(self) -> int:
        result: int = 0
        result += (self.index << 5)
        result += ((self.piece.name + 1) << 1)
        result += 1 if self.turn else 0
        return result
    
    def equal(self, target: Self) -> bool:
        return self.index == target.index
    
    def ableMove(self, address: Address) -> bool:
        if self.turn == Turn.FIRST:
            if self.piece.name == PieceName.KNIGHT and address.row < 3:
                return False
            elif (self.piece.name == PieceName.LANCE or self.piece.name == PieceName.PAWN) and address.row < 2:
                return False
        elif self.turn == Turn.SECOND:
            if self.piece.name == PieceName.KNIGHT and address.row > 7:
                return False
            elif (self.piece.name == PieceName.LANCE or self.piece.name == PieceName.PAWN) and address.row > 8:
                return False
        return True
    
    def ablePromote(self, oldAddress: Address, newAddress: Address) -> bool:
        if self.piece.promotion:
            if self.turn == Turn.FIRST:
                return oldAddress.row < 4 or newAddress.row < 4
            else:
                return oldAddress.row > 6 or newAddress.row > 6
        return False

In [11]:
class Square:
    address: Address
    isUsed: SquareStatus
    piece: Piece

    def __init__(self, address: Address, isUsed: SquareStatus, piece: Piece):
        self.address = address
        self.isUsed = isUsed
        self.piece = piece

    def deployPiece(self, piece: Piece):
        self.isUsed = SquareStatus.USED
        self.piece = piece

In [12]:
class Board:
    squares: List[Square] = [None] * NUMBER_OF_SQUARES

    def deployPiece(self, piece: Piece, row: int, column: int):
        self.squares[9 * (row - 1) + (column - 1)].deployPiece(piece)

    def __init__(self):
        for i in range(NUMBER_OF_SQUARES):
            address = Address(i / 9 + 1, i % 9 + 1)
            piece = Piece(PieceName.NON, 0, Turn.FIRST)
            square = Square(address, SquareStatus.NOT_USED, piece)
            self.squares[i] = square

        self.deployPiece(Piece(PieceName.KING, 1, Turn.FIRST), 9, 5)
        self.deployPiece(Piece(PieceName.ROOK, 2, Turn.FIRST), 8, 2)
        self.deployPiece(Piece(PieceName.BICHOP, 3, Turn.FIRST), 8, 8)
        self.deployPiece(Piece(PieceName.GOLDGENERAL, 4, Turn.FIRST), 9, 4)
        self.deployPiece(Piece(PieceName.GOLDGENERAL, 5, Turn.FIRST), 9, 6)
        self.deployPiece(Piece(PieceName.SILVERGENERAL, 6, Turn.FIRST), 9, 3)
        self.deployPiece(Piece(PieceName.SILVERGENERAL, 7, Turn.FIRST), 9, 7)
        self.deployPiece(Piece(PieceName.KNIGHT, 8, Turn.FIRST), 9, 2)
        self.deployPiece(Piece(PieceName.KNIGHT, 9, Turn.FIRST), 9, 8)
        self.deployPiece(Piece(PieceName.LANCE, 10, Turn.FIRST), 9, 1)
        self.deployPiece(Piece(PieceName.LANCE, 11, Turn.FIRST), 9, 9)
        self.deployPiece(Piece(PieceName.PAWN, 12, Turn.FIRST), 7, 1)
        self.deployPiece(Piece(PieceName.PAWN, 13, Turn.FIRST), 7, 2)
        self.deployPiece(Piece(PieceName.PAWN, 14, Turn.FIRST), 7, 3)
        self.deployPiece(Piece(PieceName.PAWN, 15, Turn.FIRST), 7, 4)
        self.deployPiece(Piece(PieceName.PAWN, 16, Turn.FIRST), 7, 5)
        self.deployPiece(Piece(PieceName.PAWN, 17, Turn.FIRST), 7, 6)
        self.deployPiece(Piece(PieceName.PAWN, 18, Turn.FIRST), 7, 7)
        self.deployPiece(Piece(PieceName.PAWN, 19, Turn.FIRST), 7, 8)
        self.deployPiece(Piece(PieceName.PAWN, 20, Turn.FIRST), 7, 9)

        self.deployPiece(Piece(PieceName.KING, 21, Turn.SECOND), 1, 5)
        self.deployPiece(Piece(PieceName.ROOK, 22, Turn.SECOND), 2, 8)
        self.deployPiece(Piece(PieceName.BICHOP, 23, Turn.SECOND), 2, 2)
        self.deployPiece(Piece(PieceName.GOLDGENERAL, 24, Turn.SECOND), 1, 4)
        self.deployPiece(Piece(PieceName.GOLDGENERAL, 25, Turn.SECOND), 1, 6)
        self.deployPiece(Piece(PieceName.SILVERGENERAL, 26, Turn.SECOND), 1, 3)
        self.deployPiece(Piece(PieceName.SILVERGENERAL, 27, Turn.SECOND), 1, 7)
        self.deployPiece(Piece(PieceName.KNIGHT, 28, Turn.SECOND), 1, 2)
        self.deployPiece(Piece(PieceName.KNIGHT, 29, Turn.SECOND), 1, 8)
        self.deployPiece(Piece(PieceName.LANCE, 30, Turn.SECOND), 1, 1)
        self.deployPiece(Piece(PieceName.LANCE, 31, Turn.SECOND), 1, 9)
        self.deployPiece(Piece(PieceName.PAWN, 32, Turn.SECOND), 3, 1)
        self.deployPiece(Piece(PieceName.PAWN, 33, Turn.SECOND), 3, 2)
        self.deployPiece(Piece(PieceName.PAWN, 34, Turn.SECOND), 3, 3)
        self.deployPiece(Piece(PieceName.PAWN, 35, Turn.SECOND), 3, 4)
        self.deployPiece(Piece(PieceName.PAWN, 36, Turn.SECOND), 3, 5)
        self.deployPiece(Piece(PieceName.PAWN, 37, Turn.SECOND), 3, 6)
        self.deployPiece(Piece(PieceName.PAWN, 38, Turn.SECOND), 3, 7)
        self.deployPiece(Piece(PieceName.PAWN, 39, Turn.SECOND), 3, 8)
        self.deployPiece(Piece(PieceName.PAWN, 40, Turn.SECOND), 3, 9)
    
    def serchPiece(self, piece: Piece) -> Address:
        for i in self.squares:
            if i.piece.equal(piece):
                return i.address
            
    def getSquareWhenMoved(self, piece: Piece, address: Address, length: int, direction: Direction) -> Square:
        d = [Direction] * MAXIMUM_NUMBER_OF_DIRECTIONS_DUE_TO_POSITIONAL_MOVEMENT
        l = [int] * MAXIMUM_NUMBER_OF_DIRECTIONS_DUE_TO_POSITIONAL_MOVEMENT

        for index, value in enumerate(piece.piece.wayOfMove[direction].adirectionMove):
            if piece.turn == Turn.FIRST:
                d[index] = value.direction
            else:
                d[index] = switchDirection(value.direction)
            l[index] = value.maxLength

        if l[0] < length: l[0] = length
        # if length > l[0]: length = l[0]
        
        vector = [Address] * 2
        vector[0] = getAddressOfDirection(d[0])
        vector[1] = getAddressOfDirection(d[1])

        address.row += vector[0].row * l[0] + vector[1].row * l[1]
        address.column += vector[0].column * l[0] + vector[1].column * l[1]

        for i in self.squares:
            if i.address.equal(address):
                return i
        
        return Square(Address(0, 0), SquareStatus.NOT_USED, Piece(PieceName.NON, 0, Turn.FIRST))

    def isTwoSteps(self, piece: Piece, address: Address) -> bool:
        if piece.piece.name == PieceName.PAWN:
            for i in self.squares:
                if i.address.column == address.column:
                    if i.piece.piece.name == PieceName.PAWN and i.piece.turn == piece.turn:
                        return True
            return False
        else:
            return False
    

In [13]:
class Condition:
    board: Board
    notOnBoard: List[Piece] = [None] * NUMBER_OF_PIECES
    turn: Turn
    turnNumber: int

    def __init__(self):
        self.board = Board()
        self.turnNumber = 1
        self.turn = Turn.FIRST
        self.initNotOnBoard()

    def initNotOnBoard(self):
        for i in range(NUMBER_OF_PIECES):
            self.notOnBoard[i] = Piece(PieceName.NON, 0, Turn.FIRST)

    def serchPieceFromNotOnBoard(self, piece: Piece) -> int:
        for i in range(NUMBER_OF_PIECES):
            if self.notOnBoard[i].equal(piece):
                return i
            
    def deployPieceNotOnBoard(self, index: int, piece: Piece):
        self.notOnBoard[index] = piece

In [14]:
class Move:
    address: Address
    piece: Piece

    def __init__(self, address: Address, piece: Piece):
        self.address = address
        self.piece = piece

In [15]:
# 関数定義  
def getPromotedPieceName(name: PieceName) -> PieceName:
    if name == PieceName.NON:
        return PieceName.NON
    elif name == PieceName.KING:
        return PieceName.NON
    elif name == PieceName.ROOK:
        return PieceName.PROMOTED_ROOK
    elif name == PieceName.BICHOP:
        return PieceName.PROMOTED_BICHOP
    elif name == PieceName.GOLDGENERAL:
        return PieceName.NON
    elif name == PieceName.SILVERGENERAL:
        return PieceName.PROMOTED_SILVERGENERAL
    elif name == PieceName.KNIGHT:
        return PieceName.PROMOTED_KNIGHT
    elif name == PieceName.LANCE:
        return PieceName.PROMOTED_LANCE
    elif name == PieceName.PAWN:
        return PieceName.PROMOTED_PAWN
    elif name == PieceName.PROMOTED_ROOK:
        return PieceName.NON
    elif name == PieceName.PROMOTED_BICHOP:
        return PieceName.NON
    elif name == PieceName.PROMOTED_SILVERGENERAL:
        return PieceName.NON
    elif name == PieceName.PROMOTED_KNIGHT:
        return PieceName.NON
    elif name == PieceName.PROMOTED_LANCE:
        return PieceName.NON
    elif name == PieceName.PROMOTED_PAWN:
        return PieceName.NON
    
def getPieceNameBeforePromote(name: PieceName) -> PieceName:
    if name == PieceName.NON:
        return PieceName.NON
    elif name == PieceName.KING:
        return PieceName.KING
    elif name == PieceName.ROOK:
        return PieceName.ROOK
    elif name == PieceName.BICHOP:
        return PieceName.BICHOP
    elif name == PieceName.GOLDGENERAL:
        return PieceName.GOLDGENERAL
    elif name == PieceName.SILVERGENERAL:
        return PieceName.SILVERGENERAL
    elif name == PieceName.KNIGHT:
        return PieceName.KNIGHT
    elif name == PieceName.LANCE:
        return PieceName.LANCE
    elif name == PieceName.PAWN:
        return PieceName.PAWN
    elif name == PieceName.PROMOTED_ROOK:
        return PieceName.ROOK
    elif name == PieceName.PROMOTED_BICHOP:
        return PieceName.BICHOP
    elif name == PieceName.PROMOTED_SILVERGENERAL:
        return PieceName.SILVERGENERAL
    elif name == PieceName.PROMOTED_KNIGHT:
        return PieceName.KNIGHT
    elif name == PieceName.PROMOTED_LANCE:
        return PieceName.LANCE
    elif name == PieceName.PROMOTED_PAWN:
        return PieceName.PAWN
    
def displayCondition(condition: Condition):
        print("|9|8|7|6|5|4|3|2|1|")
        for i in condition.board.squares:
            if i.address.column == 1: print("|", end="")
            if i.piece.piece.name == PieceName.NON:
                print(f"{i.piece.piece.pieceToString()}|", end="")
            elif i.piece.turn == Turn.FIRST:
                print(ConsoleColor.FIRST + f"{i.piece.piece.pieceToString()}" + ConsoleColor.RESET + "|", end="")
            elif i.piece.turn == Turn.SECOND:
                print(ConsoleColor.SECOND + f"{i.piece.piece.pieceToString()}" + ConsoleColor.RESET + "|", end="")
            if i.address.column == NUMBER_OF_EDGE: print("")
        
        print("NotOnBoard:")
        for i in condition.notOnBoard:
            if i.piece.name == PieceName.NON: continue
            if i.turn == Turn.FIRST:
                print(ConsoleColor.FIRST + f"{i.piece.piece.pieceToString()}" + ConsoleColor.RESET + ",", end="")
            elif i.piece.turn == Turn.SECOND:
                print(ConsoleColor.SECOND + f"{i.piece.piece.pieceToString()}" + ConsoleColor.RESET + ",", end="")
        print("")

def addMoves(moves: List[Move], address: Address, piece: Piece) -> List[Move]:
    return moves.append(Move(address, piece))

def serchAddMoves(pointableHands: List[Move], fromSquare: Square, targetSquare: Square, turn: Turn) -> List[Move]:
    if fromSquare.piece.ablePromote(fromSquare.address, targetSquare.address):
        if fromSquare.piece.ableMove(targetSquare.address):
            pointableHands = addMoves(pointableHands, targetSquare.address)
            promotedPiece: Piece = fromSquare.piece
            promotedPiece.piece.name = getPromotedPieceName(fromSquare.piece.piece.name)
            promotedPiece.piece.promotion = Promotion.NOTPROMOTABLE
            pointableHands = addMoves(pointableHands, targetSquare.address, promotedPiece)
        else:
            promotedPiece: Piece = fromSquare.piece
            promotedPiece.piece.name = getPromotedPieceName(fromSquare.piece.piece.name)
            promotedPiece.piece.promotion = Promotion.NOTPROMOTABLE
            pointableHands = addMoves(pointableHands, targetSquare.address, promotedPiece)
    else:
        pointableHands = addMoves(pointableHands, targetSquare.address, fromSquare.piece)
    return pointableHands

In [16]:
c = Condition()
displayCondition(c)

|9|8|7|6|5|4|3|2|1|
|l|n|s|g|k|g|s|n|l|
| |b| | | | | |r| |
|p|p|p|p|p|p|p|p|p|
| | | | | | | | | |
| | | | | | | | | |
| | | | | | | | | |
|p|p|p|p|p|p|p|p|p|
| |r| | | | | |b| |
|l|n|s|g|k|g|s|n|l|
NotOnBoard:

